<a href="https://colab.research.google.com/github/pxm7946/big-data-challenge/blob/main/level_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-01 06:41:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-03-01 06:41:38 (5.95 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
spark.sparkContext.addFile("https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz")

In [5]:
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [6]:
votes_df = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
votes_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            3|          3|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            1|          1|   N|                Y|
|          5|            0|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            1|          2|   N|                Y|
|          4|            1|          1|   N|                Y|
|          4|            1|          1|   N|           

In [8]:
votes_df2 = votes_df.filter(votes_df["total_votes"] >= 20)

In [10]:
votes_clean= votes_df2.filter(votes_df2["helpful_votes"]/votes_df2["total_votes"] >= 0.5)

In [11]:
from pyspark.sql.functions import col, avg

In [12]:
paid_df = votes_clean.filter(votes_clean["vine"] == 'Y')
paid_df.describe().show()

+-------+------------------+-----------------+------------------+----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+-----------------+------------------+----+-----------------+
|  count|                49|               49|                49|  49|               49|
|   mean|3.4081632653061225|55.69387755102041|61.857142857142854|null|             null|
| stddev|1.1165118178755578|91.61723002107568| 96.61305984872507|null|             null|
|    min|                 1|               11|                20|   Y|                N|
|    max|                 5|              520|               537|   Y|                Y|
+-------+------------------+-----------------+------------------+----+-----------------+



In [13]:
unpaid_df= votes_clean.filter(votes_clean["vine"] == 'N')
unpaid_df.describe().show()

+-------+------------------+-----------------+------------------+------+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|  vine|verified_purchase|
+-------+------------------+-----------------+------------------+------+-----------------+
|  count|            151400|           151400|            151400|151400|           151400|
|   mean| 3.810759577278732|46.61474900924703|53.334068692206074|  null|             null|
| stddev|1.5118976818176069|99.70198118650013|108.89142486723924|  null|             null|
|    min|                 1|               10|                20|     N|                N|
|    max|                 5|            20928|             22982|     N|                Y|
+-------+------------------+-----------------+------------------+------+-----------------+



In [14]:
paid_5stars = paid_df[paid_df["star_rating"] == 5].count()
paid_total=paid_df.count()

print(paid_5stars)
print(paid_total)

9
49


In [17]:
percentage_5star = float(paid_5stars)/float(paid_total)
print(percentage_5star)

0.1836734693877551


In [16]:
unpaid_5stars = unpaid_df[unpaid_df["star_rating"] == 5].count()
unpaid_total = unpaid_df.count()

print(unpaid_5stars)
print(unpaid_total)

78061
151400


In [18]:
percentage_5star_up = float(unpaid_5stars)/float(unpaid_total)
print(percentage_5star_up)

0.5155944517833554
